In [3]:
# this notebook contains code for extracting movie features from wikidata. here are the steps that I follow:

# 1. get a list of all the movies
# 2. run entity linking use the `find_wikidata_id` function to link them to a wikidata entity
# 3. run the two SPARQL queries (3a and 3b)
# 4. format data and write it to a csv

import re
from typing import List
from more_itertools import chunked

import mkwikidata
import pandas as pd
from tqdm import tqdm
tqdm.pandas()


from recommenders.datasets import movielens
from recommenders.datasets.wikidata import find_wikidata_id # resolve movie titles to a QID


# constants
MOVIELENS_DATA_SIZE = "100k"
WIKIDATA_BATCH_SIZE = 300
SEED = 42

In [4]:
# step 1: get a list of movies


movielens_df = movielens.load_pandas_df(
    size=MOVIELENS_DATA_SIZE,
    genres_col='genre',
    title_col='title',
    year_col='year',
    header=["userID", "itemID", "rating"]
)

movielens_df.head(5) # quick look at the data


100%|██████████| 4.81k/4.81k [00:01<00:00, 3.05kKB/s]


,userID,itemID,rating,title,genre,year
0,196,242,3.0,Kolya (1996),Comedy,1996
1,63,242,3.0,Kolya (1996),Comedy,1996
2,226,242,5.0,Kolya (1996),Comedy,1996
3,154,242,3.0,Kolya (1996),Comedy,1996
4,306,242,5.0,Kolya (1996),Comedy,1996


In [5]:
# step 2: apply entity linking to all of the titles
items_df = movielens_df[["itemID","title"]].drop_duplicates()
items_df["wikiID"] = items_df["title"].progress_apply(find_wikidata_id)

100%|██████████| 1682/1682 [08:40<00:00,  3.23it/s]


In [6]:
items_df[items_df["wikiID"] == "entityNotFound"] # we failed to find wiki IDs for only 3 movies

,itemID,title,wikiID
99626,1569,"Vie est belle, La (Life is Rosey) (1987)",entityNotFound
99795,1345,"Day the Sun Turned Cold, The (Tianguo niezi) (...",entityNotFound
99916,1634,Etz Hadomim Tafus (Under the Domin Tree) (1994),entityNotFound


In [7]:
# step 3: extract features

def sparql2pd(query: str, movie_qids: List[str], id_col: str = "item") -> pd.DataFrame:
    # this function will retrieve data from wikidata for a list of movie ids
    # i use the library `mkwikidata` to do this because python formatting strings and SPARQL do not mix well.
    qids_format = " ".join([f"wd:{qid}" for qid in movie_qids])

    
    result = mkwikidata.run_query(query, params={ "movies": qids_format })

    # assert len(result["results"]["bindings"]) == len(movie_qids) 
    # useful for ensuring queries don't have duplicate fields 
    
    cols = result["head"]["vars"]
    data = pd.DataFrame.from_records(
        { col: i[col]["value"] if col in i else "" for col in cols }
        for i in result["results"]["bindings"]
    )
    data["wikiID"] = data.pop(id_col).apply(lambda url: re.findall("Q[0-9]+",url)[0] )

    return data

# test_qids = ["Q222720","Q17738","Q153723","Q44578","Q271830"]
qids = [q for q in items_df["wikiID"].unique() if q !='entityNotFound']

In [27]:
# extracts:
# - MPA rating
# - Bechdel/mako mori P/F
# - rotten tomatoes score
query_3a = """
#title: Movie Metadata
SELECT ?item ?itemLabel ?bechdelOutcomeLabel ?makoMoriOutcomeLabel ?mpaRatingLabel ?tomatoScore
WHERE 
{
  VALUES ?item {
     $movies
  }

  OPTIONAL {           
    ?item p:P5021 ?test .    
    ?test ps:P5021	wd:Q4165246 ;  # filter for bechdel test
          pq:P9259 ?bechdelOutcome 
  }

  OPTIONAL {           
    ?item p:P5021 ?test2 .    
    ?test2 ps:P5021	wd:Q85783379 ;  # filter for mako mori test
           pq:P9259 ?makoMoriOutcome 
  }
  
  OPTIONAL {
    ?item p:P444 ?tomato .
    ?tomato pq:P459 wd:Q108403393 ;
            ps:P444 ?scoreString
    
    BIND(xsd:decimal(REPLACE(?scoreString, "%", ""))/100 AS ?tomatoScore)
  }

  OPTIONAL { ?item wdt:P1657 ?mpaRating . }   # MPA film rating
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
"""

df_3a = pd.concat( 
    sparql2pd(query_3a,qids_batch) for qids_batch in chunked(qids, WIKIDATA_BATCH_SIZE) # run queries with a few qids at a time
).reset_index(drop=True)

In [28]:
# there are some duplicate ids returned by this query
df_3a["wikiID"].value_counts().head(10)

Q168154    2
Q174284    2
Q190956    2
Q170564    2
Q659609    2
Q464032    2
Q271830    2
Q276523    2
Q193570    2
Q45354     1
Name: wikiID, dtype: int64

In [30]:
# there are not so many so let's look at them all.
dup_qids = list(df_3a["wikiID"].value_counts().head(9).index)
df_3a[df_3a["wikiID"].isin(dup_qids)]

,itemLabel,bechdelOutcomeLabel,makoMoriOutcomeLabel,mpaRatingLabel,tomatoScore,wikiID
44,Before Sunrise,fails,passes,R,1,Q659609
45,Before Sunrise,passes,passes,R,1,Q659609
126,Terminator 2: Judgment Day,fails,passes,R,0.92,Q170564
127,Terminator 2: Judgment Day,passes,passes,R,0.92,Q170564
131,Raiders of the Lost Ark,passes,,PG,0.93,Q174284
132,Raiders of the Lost Ark,fails,,PG,0.93,Q174284
145,Amadeus,passes,,PG,0.89,Q190956
146,Amadeus,passes,,R,0.89,Q190956
560,The Third Man,fails,fails,,0.99,Q271830
561,The Third Man,passes,fails,,0.99,Q271830


In [33]:
# deduplicate do this programatically by sorting in terms of 'priority' and then dropping duplicates

priority_map = {
    key: priority for priority, key in enumerate(['passes', 'fails','G','PG','PG-13','R','NC-17',''])
}

def get_priority(x) -> int:
    # rank statements according to the following rules:
    # - if there are conflicting tests, assume that it passed
    # - if multiple MPA ratings, apply the least restrictive 
    duplicate_features = ['bechdelOutcomeLabel','makoMoriOutcomeLabel','mpaRatingLabel']
    return sum([ priority_map.get(x[i], len(priority_map)) for i in duplicate_features ])    

    
df_3a["priority"] = df_3a.apply(get_priority,axis=1)
df_3a = df_3a.sort_values(by=["priority"],ascending=True).drop_duplicates(subset=['wikiID','itemLabel'],keep="first")
df_3a.pop('priority')

df_3a[df_3a["wikiID"].isin(dup_qids)]

618      2
619      2
334      2
328      2
630      2
        ..
1333    21
1334    21
1335    21
1475    21
1661    21
Name: priority, Length: 1653, dtype: int64

In [36]:
# extracts: 
# - movie duration
# - country of origin
# - count of academy awards
# - count of academy award nominations
query_3b = """
#title: movie metadata that requires a GROUP BY statement
SELECT 
  ?item
  ( MAX(?duration) as ?maxDuration )
  ( GROUP_CONCAT(DISTINCT ?originCountryLabel; SEPARATOR = "|") AS ?origin)
  ( COUNT(DISTINCT ?award) AS ?academyAwardCount)
  ( COUNT(DISTINCT ?nomination) AS ?academyNominationCount)

WHERE
{ 
  VALUES ?item {
    $movies
  }
  
  OPTIONAL { ?item wdt:P2047 ?duration }   # movie length (there are sommetimes multiple)  
  
  OPTIONAL {
    # origin country
    ?item wdt:P495 ?originCountry .
    ?originCountry rdfs:label ?originCountryLabel
    FILTER(LANG(?originCountryLabel) = "en")
  }
  
  OPTIONAL {
    # academy nominations
    ?item p:P1411 ?nomination .
    FILTER EXISTS {
      ?nomination ps:P1411 ?nominationType .
      ?nominationType wdt:P31 wd:Q19020 
    } 
  }
  
  OPTIONAL {
    # academy awards
    ?item p:P166 ?award .
    FILTER EXISTS {
      ?award ps:P166 ?awardType .
      ?awardType wdt:P31 wd:Q19020 
    } 
  }

  
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
GROUP BY ?item ?itemLabel
"""

df_3b = pd.concat( 
    sparql2pd(query_3b,qids_batch) for qids_batch in chunked(qids, WIKIDATA_BATCH_SIZE) # run queries with a few qids at a time
).reset_index(drop=True)

In [44]:
len(qids), len(df_3b["wikiID"]),df_3b["wikiID"].nunique() # no duplicate issue this time because of the GROUP BY statement


(1653, 1653, 1653)

In [46]:
# step 4: join and clean up

movie_features_all = df_3a.merge(df_3b,on="wikiID")
movie_features_all.pop("itemLabel")
out = items_df.merge(movie_features_all,on="wikiID",how="left")


In [47]:
out.sort_values(by="itemID").to_csv(
    "data/items_movielens_{size}.csv".format(size=MOVIELENS_DATA_SIZE),
    index=False
)